In [1]:
import FINE as fn
import numpy as np
import pandas as pd
import warnings
warnings.resetwarnings()

%matplotlib inline  
%load_ext autoreload
%autoreload 2

In [2]:
numberOfTimeSteps = 4
hoursPerTimeStep = 2190

In [3]:
# Create an energy system model instance 
esM = fn.EnergySystemModel(locations={'ElectrolyzerLocation', 'IndustryLocation'}, 
                            commodities={'electricity', 'hydrogen'}, 
                            numberOfTimeSteps=numberOfTimeSteps,
                            commodityUnitsDict={'electricity': r'kW$_{el}$', 'hydrogen': r'kW$_{H_{2},LHV}$'},
                            hoursPerTimeStep=hoursPerTimeStep, costUnit='1 Euro', 
                            lengthUnit='km', 
                            verboseLogLevel=0)

# time step length [h]
timeStepLength = numberOfTimeSteps * hoursPerTimeStep


### Buy electricity at the electricity market
costs = pd.DataFrame([np.array([ 0.05, 0., 0.1, 0.051,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
revenues = pd.DataFrame([np.array([ 0., 0.01, 0., 0.,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
maxpurchase = pd.DataFrame([np.array([1e6, 1e6, 1e6, 1e6,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep
esM.add(fn.Source(esM=esM, name='Electricity market', commodity='electricity', 
                    hasCapacityVariable=False, operationRateMax = maxpurchase,
                    commodityCostTimeSeries = costs,
                    commodityRevenueTimeSeries = revenues,  
                    )) # eur/kWh

### Electrolyzers
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit=r'kW$_{el}$',
                        commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                        hasCapacityVariable=True, 
                        investPerCapacity=500, # euro/kW
                        opexPerCapacity=500*0.025, 
                        interestRate=0.08,
                        economicLifetime=10))

### Hydrogen filled somewhere
esM.add(fn.Storage(esM=esM, name='Pressure tank', commodity='hydrogen',
                    hasCapacityVariable=True, capacityVariableDomain='continuous',
                    stateOfChargeMin=0.33, 
                    investPerCapacity=0.5, # eur/kWh
                    interestRate=0.08,
                    economicLifetime=30))

### Hydrogen pipelines
esM.add(fn.Transmission(esM=esM, name='Pipelines', commodity='hydrogen',
                        hasCapacityVariable=True,
                        investPerCapacity=0.177, 
                        interestRate=0.08,
                        economicLifetime=40))

### Industry site
demand = pd.DataFrame([np.array([0., 0., 0., 0.,]), np.array([6e3, 6e3, 6e3, 6e3,]),],
                index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep
esM.add(fn.Sink(esM=esM, name='Industry site', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix = demand,
                ))

The distances of a component are set to a normalized value of 1.


In [4]:
esM.cluster(numberOfTypicalPeriods=1, numberOfTimeStepsPerPeriod=4, storeTSAinstance=True,
            segmentation=True, numberOfSegmentsPerPeriod=1, clusterMethod='hierarchical',
            sortValues=False, rescaleClusterPeriods=False)


Clustering time series data with 1 typical periods and 4 time steps per period...
		(0.1093 sec)



In [5]:
esM.tsaInstance.typicalPeriods

,,,Electricity market_commodityCostTimeSeries_ElectrolyzerLocation,Electricity market_commodityCostTimeSeries_IndustryLocation,Electricity market_commodityRevenueTimeSeries_ElectrolyzerLocation,Electricity market_commodityRevenueTimeSeries_IndustryLocation,Electricity market_operationRate_ElectrolyzerLocation,Electricity market_operationRate_IndustryLocation,Industry site_operationRate_ElectrolyzerLocation,Industry site_operationRate_IndustryLocation
,Segment Step,Segment Duration,,,,,,,,
0,0,4,0.05025,0.0,0.0025,0.0,2.190000e+09,0.0,0.0,13140000.0


In [6]:
esM.periodsOrder

array([0])

In [7]:
esM.optimize(timeSeriesAggregation=True, solver = 'gurobi')

Time series aggregation specifications:
Number of typical periods:1, number of time steps per period:4, number of segments per period:1

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.6717 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0000 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0156 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0000 sec)

Declaring shared potential constraint...
		(0.0000 sec)

Declaring commodity balances...
		(0.0000 sec)

Declaring objective function...
		(0.0000 sec)

Using license file C:\Users\m.hoffmann\gurobi.lic
Academic license - for non-commercial use only
Read LP format

In [8]:
esM.pyM.Obj()

1642265.4396218362

In [9]:
np.testing.assert_almost_equal(3883295.265696861, esM.pyM.Obj())

AssertionError: 
Arrays are not almost equal to 7 decimals
 ACTUAL: 3883295.265696861
 DESIRED: 1642265.4396218362

In [ ]:
esM.pyM.timeSet.pprint()

In [ ]:
esM.pyM.interTimeStepsSet.pprint()